In [1]:
# Import libs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
import time
import os
import PIL.Image as Image
from IPython.display import display
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
dataset_dir_train = "../input/training-dataset-v6/finalDataset"

train_transform = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                 transforms.RandomRotation(20),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])


dataset = torchvision.datasets.ImageFolder(root=dataset_dir_train, transform = train_transform)
trainloader = torch.utils.data.DataLoader(dataset, batch_size = 32, shuffle=True, num_workers = 2)

In [4]:
# function to train the model
def TrainModel(model, lossFun, optimizer, scheduler, n_epochs=5):
    losses = []
    accuracies = []
    
    model.train()
    
    for epoch in range(n_epochs):
        print(epoch)
        since = time.time() # record the start time
        running_loss = 0.0
        running_correct = 0.0
        counter = 0
        for i, data in enumerate(trainloader, 0):
            
            inputs, labels = data
            # Asigning GPU
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            counter += len(inputs)
            outputs = model(inputs)
            
            m = nn.LogSoftmax(dim=1)
            _, predicted = torch.max(outputs.data, 1)
            loss = lossFun(m(outputs), labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            running_correct += (labels==predicted).sum().item()

        timeSpent = time.time()-since # time used
        epoch_loss = running_loss/counter
        print(running_correct, counter)
        epoch_acc = 100*(running_correct/counter)
        print("Epoch %s, Time Used: %d s, loss: %.4f, acc: %.4f" % (epoch+1, timeSpent, epoch_loss, epoch_acc))
        
        losses.append(epoch_loss)
        accuracies.append(epoch_acc)
        scheduler.step(epoch_acc)
        
        torch.save(model, str(epoch) + "_epoch.pt")
    print('Training Finished')
    return model, losses, accuracies

In [5]:
regnet_v6 = torch.load("../input/models/regnet_y_3_2gf_v6.pt", torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))
regnet_v4 = torch.load("../input/models/regnet_y_3_2gf_v4.pt", torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))
regnet_1000x400 = torch.load("../input/models/regnet_y_3_2gf_v(1000 x 400 x 23).pt", torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))

In [6]:
regnet_v6.eval()
regnet_v4.eval()
regnet_1000x400.eval()
print("---")

---


In [7]:
class EnsembleModel(nn.Module):   
    def __init__(self, modelA, modelB, modelC, modelD):
        super().__init__()
        self.modelA = modelA
        self.modelB = modelB
        self.modelC = modelC
        self.classifier = modelD
        
    def forward(self, x):
        x1 = self.modelA(x)
        x2 = self.modelB(x)
        x3 = self.modelC(x)
        
        x1 = x1.to('cpu').numpy()
        x2 = x2.to('cpu').numpy()
        x3 = x3.to('cpu').numpy()
        
        x = []
        for i in range(len(x1)):
            temp = [[x1[i]], [x2[i]], [x3[i]]]
            x.append(temp)
        x = torch.tensor(x)
        x = x.to("cuda:0")
        out = self.classifier(x)
        return out

In [8]:
model = models.regnet_y_3_2gf(pretrained = True)

Downloading: "https://download.pytorch.org/models/regnet_y_3_2gf-b5a9779c.pth" to /root/.cache/torch/hub/checkpoints/regnet_y_3_2gf-b5a9779c.pth


  0%|          | 0.00/74.6M [00:00<?, ?B/s]

In [9]:
model.fc = nn.Sequential(
    model.fc,
    nn.Linear(in_features=1000, out_features=400, bias = True),
    nn.Linear(in_features=400, out_features=150, bias=True),
    nn.Linear(in_features=150, out_features=23, bias=True)
)

model = model.to(device)

In [10]:
ensemble_model = EnsembleModel(regnet_v6, regnet_v4, regnet_1000x400, model)

for param in ensemble_model.modelA.parameters():
    param.requires_grad = False

for param in ensemble_model.modelB.parameters():
    param.requires_grad = False

for param in ensemble_model.modelC.parameters():
    param.requires_grad = False
    
for param in ensemble_model.classifier.parameters():
    param.requires_grad = True

ensemble_model = ensemble_model.to("cuda:0" if torch.cuda.is_available() else "cpu")

In [11]:
lossFun = nn.NLLLoss()
optimizer = optim.SGD(ensemble_model.parameters(), lr=0.008, momentum=0.9)
lrscheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=2, threshold = 0.9)

In [12]:
model, training_losses, training_accs = TrainModel(ensemble_model, lossFun, optimizer, lrscheduler, n_epochs=25)

0
43085.0 46374
Epoch 1, Time Used: 659 s, loss: 0.0078, acc: 92.9077
1
45115.0 46374
Epoch 2, Time Used: 586 s, loss: 0.0026, acc: 97.2851
2
45341.0 46374
Epoch 3, Time Used: 583 s, loss: 0.0022, acc: 97.7725
3
45452.0 46374
Epoch 4, Time Used: 593 s, loss: 0.0019, acc: 98.0118
4
45599.0 46374
Epoch 5, Time Used: 596 s, loss: 0.0015, acc: 98.3288
5
45650.0 46374
Epoch 6, Time Used: 595 s, loss: 0.0013, acc: 98.4388
6
45674.0 46374
Epoch 7, Time Used: 601 s, loss: 0.0013, acc: 98.4905
7
45720.0 46374
Epoch 8, Time Used: 603 s, loss: 0.0013, acc: 98.5897
8
45718.0 46374
Epoch 9, Time Used: 595 s, loss: 0.0012, acc: 98.5854
9
45730.0 46374
Epoch 10, Time Used: 596 s, loss: 0.0012, acc: 98.6113
10
45691.0 46374
Epoch 11, Time Used: 595 s, loss: 0.0013, acc: 98.5272
11
45719.0 46374
Epoch 12, Time Used: 598 s, loss: 0.0012, acc: 98.5876
12
45702.0 46374
Epoch 13, Time Used: 597 s, loss: 0.0013, acc: 98.5509
13
45710.0 46374
Epoch 14, Time Used: 596 s, loss: 0.0012, acc: 98.5682
14
45678.0 

In [13]:
model.eval();
torch.save(model, "final.pt");